In [ ]:
import pandas as pd

Import data to a pandas data frame

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train_data = train.copy()
test_data = test.copy()

looking to data 

In [ ]:
train_data.head()

Dropping the NaN values

In [ ]:
train_data = train_data.set_index('id', drop = True)

In [ ]:
print(train_data.shape)
train_data.head()

In [ ]:
print(test_data.shape)
test_data.head()

In [ ]:
# checking for missing values
train_data.isnull().sum()

In [ ]:
# dropping missing values from text columns alone. 
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

In [ ]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

In [ ]:
min(train_data['length']), max(train_data['length']), round(sum(train_data['length'])/len(train_data['length']))

In [ ]:
len(train_data[train_data['length'] < 50])

In [ ]:
train_data['text'][train_data['length'] < 50]

In [ ]:
# dropping the outliers
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)

In [ ]:
min(train_data['length']), max(train_data['length']), round(sum(train_data['length'])/len(train_data['length']))

In [ ]:
##importing tensorflow and looking into the version of it
import tensorflow as tf
tf.__version__

In [ ]:
## import all necessaries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout, RNN, SpatialDropout1D

In [ ]:
## voc size
voc_size =  4500

In [ ]:
tokenizer = Tokenizer(num_words = voc_size, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [ ]:
# now applying padding to make them even shaped.
X = pad_sequences(sequences = X, maxlen = voc_size, padding = 'pre')

In [ ]:
print(X.shape)
y = train_data['label'].values
print(y.shape)

In [ ]:
# splitting the data training data for training and validation.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

**Lets create our model......**

In [ ]:
##creating model
embdding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embdding_vector_features, input_length = 20))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics =  ['accuracy'])
print(model.summary())

# Woohooo!!! lets train our model....

In [ ]:
## model training 
model_fit = model.fit(X_train, y_train, epochs = 1)

Performance matrix and accuracy

In [ ]:
print(test.shape)
test_data = test.copy()
print(test_data.shape)

In [ ]:
test_data = test_data.set_index('id', drop = True)
test_data.shape

In [ ]:
test_data = test_data.fillna(' ')
print(test_data.shape)
test_data.isnull().sum()

In [ ]:
tokenizer.fit_on_texts(texts = test_data['text'])
test_text = tokenizer.texts_to_sequences(texts = test_data['text'])

In [ ]:
test_text = pad_sequences(sequences = test_text, maxlen = voc_size, padding = 'pre')


In [ ]:
pred = model.predict_classes(test_text)

In [ ]:
len(test_data['id']), len(pred)


In [ ]:
df_sub = pd.DataFrame()
df_sub['id'] = test_data['id']
df_sub['label'] = pred

In [ ]:

df_sub.to_csv('submission1.csv', index=False)

In [ ]:
pred

In [ ]:
df_sub.head()